# Pre

In [1]:
from __future__ import annotations
%pip install geopy
%pip install python-dotenv
%pip install pyshp
%pip install --upgrade --force-reinstall \
    mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
try:
    from mediocreatbest import auto
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
    from mediocreatbest import auto

from opencage.geocoder import OpenCageGeocode

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git to /tmp/pip-install-yycqp1fr/mediocreatbest_89a4d62a4e1f4b169702a3ab7fb99cc

# Config

In [2]:
auto.dotenv.load_dotenv()
config = auto.types.SimpleNamespace()

config.opencage = auto.types.SimpleNamespace()
config.opencage.token = auto.os.getenv('OPENCAGE_API_KEY')

config.map = auto.types.SimpleNamespace()
config.map.token = auto.os.getenv('MAPBOX_API_KEY')

config.buildings = auto.types.SimpleNamespace()
config.buildings.datadir = auto.pathlib.Path('/mnt/seenas2/data/model-america/data')
config.buildings.csvs = config.buildings.datadir / 'MAv1_CSVS'
config.buildings.gendir = auto.pathlib.Path('data/gen')
assert config.buildings.datadir.exists()
assert config.buildings.csvs.exists()
assert config.buildings.gendir.exists()


# Lib

## Polygon Containment

In [3]:
def is_point_in_polygon(point, polygon):
    x, y = point
    inside = False
    
    # Get the number of vertices in the polygon
    n = len(polygon)
    
    # Store the first and last point of the polygon
    p1x, p1y = polygon[0]
    
    # Iterate through each edge of the polygon
    for i in range(n + 1):
        # Get the next point (wrapping around to the first point when needed)
        p2x, p2y = polygon[i % n]
        
        # Check if the point is within the vertical range of the edge
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                # If the point is within the horizontal range, calculate the x-intercept
                if x <= max(p1x, p2x):
                    # Calculate the x-intercept of the ray
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    
                    # If the point is on the right side of the x-intercept, toggle inside
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        
        # Move to the next edge
        p1x, p1y = p2x, p2y
    
    return inside


## MapUtil

In [4]:
class MapUtil:
    token:str = None
    geocoder = None

    @classmethod
    def __init__(self, token=config.opencage.token):
        self.token = token
        self.geocoder = auto.opencage.geocoder.OpenCageGeocode(self.token)
    
    @classmethod
    def addr_to_state(cls, addr: str) -> str:
        data = cls.geocoder.geocode(addr)[0]
        state = data['components']
        return cls.state_abbreviation(state.get('state'))

    @classmethod
    def coords_to_addr(cls, lat=None, lon=None):
        url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key={cls.token}"
        response = auto.requests.get(url)
        data = response.json()
        if data['results']:
            address = data['results'][0]['formatted']
            return address
        else:
            return None
            print("No results found.")

    @classmethod
    def addr_to_coords(cls, addr:str) -> (float, float):
        data = cls.geocoder.geocode(addr)[0]
        lat = data['annotations']['DMS']['lat']
        lng = data['annotations']['DMS']['lng']
        lat = cls.dms_to_decimal(lat)
        lng = cls.dms_to_decimal(lng)
        return (lat, lng)

    @staticmethod
    def get_point_at_distance(lat, lon, distance_miles, bearing_degrees):
        lat_rad = auto.math.radians(lat)
        lon_rad = auto.math.radians(lon)
        bearing_rad = auto.math.radians(bearing_degrees)
        
        R = 3959 # Earth radius in miles
        
        d = distance_miles / R
        
        new_lat_rad = auto.math.asin(
            auto.math.sin(lat_rad) * auto.math.cos(d) +
            auto.math.cos(lat_rad) * auto.math.sin(d) * auto.math.cos(bearing_rad)
        )
        
        new_lon_rad = lon_rad + auto.math.atan2(
            auto.math.sin(bearing_rad) * auto.math.sin(d) * auto.math.cos(lat_rad),
            auto.math.cos(d) - auto.math.sin(lat_rad) * auto.math.sin(new_lat_rad)
        )
        
        new_lat = auto.math.degrees(new_lat_rad)
        new_lon = auto.math.degrees(new_lon_rad)
        
        return (new_lat, new_lon)

    @staticmethod
    def state_abbreviation(state_name: str):
        state_abbreviations = {
            'alabama': 'AL', 'alaska': 'AK', 'arizona': 'AZ', 'arkansas': 'AR',
            'california': 'CA', 'colorado': 'CO', 'connecticut': 'CT', 'delaware': 'DE',
            'florida': 'FL', 'georgia': 'GA', 'hawaii': 'HI', 'idaho': 'ID',
            'illinois': 'IL', 'indiana': 'IN', 'iowa': 'IA', 'kansas': 'KS',
            'kentucky': 'KY', 'louisiana': 'LA', 'maine': 'ME', 'maryland': 'MD',
            'massachusetts': 'MA', 'michigan': 'MI', 'minnesota': 'MN', 'mississippi': 'MS',
            'missouri': 'MO', 'montana': 'MT', 'nebraska': 'NE', 'nevada': 'NV',
            'new hampshire': 'NH', 'new jersey': 'NJ', 'new mexico': 'NM',
            'new york': 'NY', 'north carolina': 'NC', 'north dakota': 'ND',
            'ohio': 'OH', 'oklahoma': 'OK', 'oregon': 'OR', 'pennsylvania': 'PA',
            'rhode island': 'RI', 'south carolina': 'SC', 'south dakota': 'SD',
            'tennessee': 'TN', 'texas': 'TX', 'utah': 'UT', 'vermont': 'VT',
            'virginia': 'VA', 'washington': 'WA', 'west virginia': 'WV',
            'wisconsin': 'WI', 'wyoming': 'WY'
        }
        
        return state_abbreviations.get(state_name.lower().strip(), None)

    @staticmethod
    def dms_to_decimal(dms_str):
        # Regular expression to parse the DMS format
        match = auto.re.match(r"(\d+)° (\d+)' ([\d.]+)'' ([NSEW])", dms_str)
        if not match:
            raise ValueError("Invalid DMS format")

        degrees = int(match.group(1))
        minutes = int(match.group(2))
        seconds = float(match.group(3))
        direction = match.group(4)

        # Convert to decimal degrees
        decimal_degrees = degrees + minutes / 60 + seconds / 3600

        # Adjust sign for South and West
        if direction in 'SW':
            decimal_degrees *= -1

        return decimal_degrees
    
    @staticmethod
    def zoom_from_radius(radius_miles: float):
        EARTH_CIRCUMFERENCE_MILES = 24901  # Earth's circumference in miles
        TILE_SIZE = 256  # Tile size in pixels (standard Web Mercator tiles)
        BASE_RESOLUTION = EARTH_CIRCUMFERENCE_MILES / TILE_SIZE  # Resolution at zoom level 0
        
        zoom_level = auto.math.log2(EARTH_CIRCUMFERENCE_MILES / (radius_miles * 2))
        
        zoom_level = max(0, min(18, zoom_level))
        
        return round(zoom_level)

    @staticmethod
    def point_distance(lat1:float, lon1:float, lat2:float, lon2:float) -> float:
        return auto.haversine.haversine((lat1, lon1), (lat2, lon2), unit=auto.haversine.Unit.MILES)

def scope():
    MapUtil()
    addr = '8000 Middlebrook Pike Knoxville TN'
    addr2 = '1131 West Nokomis Circle Knoxville TN'
    lat, lon = MapUtil.addr_to_coords(addr)
    print(lat, lon)
    print(MapUtil.coords_to_addr(lat, lon))
    print(MapUtil.addr_to_state(addr))
    print(MapUtil.zoom_from_radius(0.03))

    lat2, lon2 = MapUtil.addr_to_coords(addr2)
    print(MapUtil.point_distance(lat, lon, lat2, lon2))

/scope

35.943011 -84.05265299999999
8000 Middlebrook Pike, Knoxville, TN 37909, United States of America
TN
18
4.8772193126856545


## Colorscheme

In [5]:
class ColorType(auto.enum.Enum):
    HEX=1
    RGB=2

class Colormap:
    cmap=None

    def __init__(self, name: str, output_type=ColorType.HEX):
        self.cmap = auto.plt.get_cmap(name)

    def colors_from_values(self, values, min=None, max=None):
        minimum = min(values) if min is None else min
        maximum = max(values) if max is None else max

        normalized = auto.matplotlib.colors.Normalize(vmin=minimum, vmax=maximum)

        colors = self.cmap(normalized(values))
        match output_type:
            case ColorType.HEX:
                colors= [auto.matplotlib.colors.to_hex(color) for color in colors]
            case ColorType.RGB:
                colors= [auto.matplotlib.colors.to_rgb(color) for color in colors]

        return colors
    
    def sm_from_values(self, values):
        minimum = min(values)
        maximum = max(values)
        normalized = auto.matplotlib.colors.Normalize(vmin=minimum, vmax=maximum)
        colors = self.cmap(normalized(values))
        sm = auto.matplotlib.cm.ScalarMappable(cmap=self.cmap, norm=normalized)
        sm.set_array([])
        return sm

## Parallel

In [6]:
@auto.dataclasses.dataclass
class TaskResult:
    task_id: int
    result: Any
    execution_time: float
    success: bool
    error: Exception = None

class ThreadExecutor:
    def __init__(self, max_workers:int=4):
        self.max_workers = max_workers
        self.executor = auto.concurrent.futures.ThreadPoolExecutor(max_workers=max_workers)

    def _execute_task(self, id:int, task: Callable, *args, **kwargs) -> TaskResult:
        start = auto.time.time()
        try:
            result = task(*args, **kwargs)
            success = True
            error = None
        except Exception as e:
            result = None
            success = False
            error = e
        
        runtime = auto.time.time() - start
        return TaskResult(
            task_id=id,
            result=result,
            execution_time=runtime,
            success=success,
            error=error
        )

    def execute(self, tasks: List[tuple[Callable, tuple, dict]]) -> List[TaskResult]:
        futures = [
            self.executor.submit(
                self._execute_task,
                task_id,
                func,
                *task_args,
                **task_kwargs
            ) for task_id, (func, task_args, task_kwargs) in enumerate(tasks)
        ]
        results = [future.result() for future in futures]
        return results


    def execute_single(self, task: Callable, *args, **kwargs) -> TaskResult:
        return self._execute_task(0, task, *args, **kwargs)

## Buildings

## Map

In [7]:
class Map:
    center: (float, float) = None
    L = None
    tile_layers = {
        'nasa': auto.ipyleaflet.basemap_to_tiles(auto.ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-04-08"),
    }

    mapbox_styles = {
        'satellite': 'https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12/tiles/{{z}}/{{x}}/{{y}}?access_token={access_token}',
        'dark': 'https://api.mapbox.com/styles/v1/mapbox/dark-v11/tiles/{{z}}/{{x}}/{{y}}?access_token={access_token}',
        'night': 'https://api.mapbox.com/styles/v1/mapbox/navigation-night-v1/tiles/{{z}}/{{x}}/{{y}}?access_token={access_token}',
    }
    
    def __init__(self, center=None, zoom=None):
        self.center = center
        self.L = auto.ipyleaflet.Map(center=self.center, zoom=zoom)
        # self.L.add(self.tile_layers['nasa'])

    def use_mapbox(self, name, api_key):
        tiles = auto.ipyleaflet.TileLayer(
            url=self.mapbox_styles[name].format(access_token=api_key),
            attribution='Mapbox'
        )
        self.L.add_layer(tiles)
        return self
    
    def add_polygon(self, polygon):
        self.L.add_layer(polygon)

    def display(self):
        /display self.L

### Preprocess buildings

In [8]:
def fmt(point_str: str) -> (float, float):
    return (
        float(point_str.split('/')[0])
        ,float(point_str.split('/')[1])
    )

def process(state, inpath):
    print(f'Processing {state}...')
    start = auto.time.time()
    outname = f'{state}_gen.csv'
    genpath = config.buildings.gendir


    out = []
    indf = auto.pd.read_csv(inpath / f'{state}.csv')

    out = []
    for row in indf.iterrows():
        try:
            centroid = fmt(row[1]['Centroid'])
            footprint = [fmt(p) for p in row[1]['Footprint2D'].split('_')] if not '' in row[1]['Footprint2D'].split('_') else [fmt(row[1]['Centroid'])]
            out.append({
                'state': row[1]['State_Abbr'],
                'centroid': centroid,
                'footprint': footprint,
            })
        except:
            pass

    # # outdf = auto.pd.DataFrame(out)
    # # outdf.to_csv(genpath / outname)
    # # print(f'Done. Took {auto.time.time() - start} seconds')
    # print(auto.termcolor.colored(f'{state} done', 'green'))
    return out
    

def scope():
    # process('AR', config.buildings.csvs)
    tasks = []
    for state in [
        "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
        "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
        "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
        "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
        "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
    ]:
        tasks.append(
            (process, (state, config.buildings.csvs), {})
        )
    threader = ThreadExecutor(max_workers=12)
    results = threader.execute(tasks)

    out = []
    print(results)
    for res in results:
        if res.success:
            print('slkdjfsdlkf')
            out.extend(res.result)

    df = auto.pd.DataFrame(out)
    /display df
    df.to_csv(config.buildings.gendir / 'buildings.csv')

# This takes long as shit so only uncomment this if we need to rerun the entire preprocessing step
# /scope

### Buildings

In [9]:
class Buildings:
    data = auto.pd.DataFrame

    def __init__(self, data):
        self.data = data

    def display(self):
        /display self.data
    

In [10]:
class BuildingsManager:
    raw: auto.pd.DataFrame

    def __init__(self, path):
        self._load_buildings(path)
        self.states = {}

    def _load_buildings(self, path):
        print(f'Opening {path}...', end='')
        self.raw = auto.pd.read_csv(path)
        print('Done.')

    def get_state(self, state: str):
        return Buildings(data=self.raw[self.raw['state'] == state])

## TODOS:
- Map data onto buildings
- Map data onto streets
- Create streets from buildings

## Roads

In [11]:
class Roads:
    gdf = None
    state_roads = {}
    def __init__(self, path):
        self.gdf = auto.geopandas.read_file(path)
        self.state_roads = {}

    def get_state(self, state: str):
        self.state_roads[state] = self.gdf[self.gdf['JURISNAME'] == state]

    def plot(self):
        self.gdf.plot()
        auto.plt.title("Shapefile Geometry")
        auto.plt.xlabel("Longitude")
        auto.plt.ylabel("Latitude")
        auto.plt.show()
    
    def plot_state(self, state: str):
        if not state in self.state_roads:
            self.get_state(state)
        
        self.state_roads[state].plot()
        auto.plt.title("Shapefile Geometry")
        auto.plt.xlabel("Longitude")
        auto.plt.ylabel("Latitude")
        auto.plt.show()


def scope():
    roads = Roads('zip:///mnt/seenas2/data/byod-roads-shapefile.zip')
    roads.plot()
    roads.plot_state('Tennessee')

    return roads

# roads = scope()


In [12]:
def scope():
    roads.plot_state('Florida')

# /scope

# App

### API

``` python
# Coordinates -> (lat, lon)
# |------------------------------------
# | Rating |      Coordinates         |
# |------------------------------------
# |   2.0  |   (-35.445, -131.550)    |
# |------------------------------------
# |   2.3  |   (-35.445, -131.551)    |
# |------------------------------------
# |   1.9  |   (-35.445, -131.552)    |
# |------------------------------------
# |   4.0  |   (-35.555, -131.551)    |
# |------------------------------------
# |   3.5  |   (-35.444, -131.552)    |
# |------------------------------------
```

``` python
# Coordinates -> (lat, lon)
# |-----------------------------------------------------------------------------|
# | Rating |                          AreaCoordinates                           |
# |-----------------------------------------------------------------------------|
# |   2.0  |   [(-35.445, -131.550),  (-35.445, -131.550), (-35.445, -131.550)] |
# |-----------------------------------------------------------------------------|
```

```python
%pip install covmis

# This uses the 'Coordinates' column as the locations for the attributes
building_data = comvis.Dataset(my_data, attribute='Rating', locations='Coordinates', maptype='building')

# This uses the 'Coordinates' column as the locations for the attributes
street_data = comvis.Dataset(my_data, attribute='Rating', locations='Coordinates', maptype='street')

# This uses the 'Coordinates' column as the locations for the attributes
area_data = comvis.Dataset(my_area_data, attribute='Rating', area_locations='AreaCoordinates', maptype='area')
```

## Example App without any data

## Test Data for buildings: providers

In [13]:
def load_providers():
    providers = auto.pandas.read_csv('/mnt/seenas2/data/healthcare_providers/Providers.csv')

    /print type(providers)

    /auto.pprint.pp providers.columns.to_list()

    /auto.pprint.pp providers.dtypes.to_dict()

    with auto.pd.option_context('display.max_columns', None):
        /display providers

    with auto.pd.option_context('display.max_columns', None):
        /display providers.describe()
    return providers

providers_data = load_providers()

<class 'pandas.core.frame.DataFrame'>
['CMS Certification Number (CCN)',
 'Provider Name',
 'Provider Address',
 'City/Town',
 'State',
 'ZIP Code',
 'Telephone Number',
 'Provider SSA County Code',
 'County/Parish',
 'Ownership Type',
 'Number of Certified Beds',
 'Average Number of Residents per Day',
 'Average Number of Residents per Day Footnote',
 'Provider Type',
 'Provider Resides in Hospital',
 'Legal Business Name',
 'Date First Approved to Provide Medicare and Medicaid Services',
 'Affiliated Entity Name',
 'Affiliated Entity ID',
 'Continuing Care Retirement Community',
 'Special Focus Status',
 'Abuse Icon',
 'Most Recent Health Inspection More Than 2 Years Ago',
 'Provider Changed Ownership in Last 12 Months',
 'With a Resident and Family Council',
 'Automatic Sprinkler Systems in All Required Areas',
 'Overall Rating',
 'Overall Rating Footnote',
 'Health Inspection Rating',
 'Health Inspection Rating Footnote',
 'QM Rating',
 'QM Rating Footnote',
 'Long-Stay QM Rating',

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Telephone Number,Provider SSA County Code,County/Parish,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Average Number of Residents per Day Footnote,Provider Type,Provider Resides in Hospital,Legal Business Name,Date First Approved to Provide Medicare and Medicaid Services,Affiliated Entity Name,Affiliated Entity ID,Continuing Care Retirement Community,Special Focus Status,Abuse Icon,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,With a Resident and Family Council,Automatic Sprinkler Systems in All Required Areas,Overall Rating,Overall Rating Footnote,Health Inspection Rating,Health Inspection Rating Footnote,QM Rating,QM Rating Footnote,Long-Stay QM Rating,Long-Stay QM Rating Footnote,Short-Stay QM Rating,Short-Stay QM Rating Footnote,Staffing Rating,Staffing Rating Footnote,Reported Staffing Footnote,Physical Therapist Staffing Footnote,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Total number of nurse staff hours per resident per day on the weekend,Registered Nurse hours per resident per day on the weekend,Reported Physical Therapist Staffing Hours per Resident Per Day,Total nursing staff turnover,Total nursing staff turnover footnote,Registered Nurse turnover,Registered Nurse turnover footnote,Number of administrators who have left the nursing home,Administrator turnover footnote,Nursing Case-Mix Index,Nursing Case-Mix Index Ratio,Case-Mix Nurse Aide Staffing Hours per Resident per Day,Case-Mix LPN Staffing Hours per Resident per Day,Case-Mix RN Staffing Hours per Resident per Day,Case-Mix Total Nurse Staffing Hours per Resident per Day,Case-Mix Weekend Total Nurse Staffing Hours per Resident per Day,Adjusted Nurse Aide Staffing Hours per Resident per Day,Adjusted LPN Staffing Hours per Resident per Day,Adjusted RN Staffing Hours per Resident per Day,Adjusted Total Nurse Staffing Hours per Resident per Day,Adjusted Weekend Total Nurse Staffing Hours per Resident per Day,Rating Cycle 1 Standard Survey Health Date,Rating Cycle 1 Total Number of Health Deficiencies,Rating Cycle 1 Number of Standard Health Deficiencies,Rating Cycle 1 Number of Complaint Health Deficiencies,Rating Cycle 1 Health Deficiency Score,Rating Cycle 1 Number of Health Revisits,Rating Cycle 1 Health Revisit Score,Rating Cycle 1 Total Health Score,Rating Cycle 2 Standard Health Survey Date,Rating Cycle 2 Total Number of Health Deficiencies,Rating Cycle 2 Number of Standard Health Deficiencies,Rating Cycle 2 Number of Complaint Health Deficiencies,Rating Cycle 2 Health Deficiency Score,Rating Cycle 2 Number of Health Revisits,Rating Cycle 2 Health Revisit Score,Rating Cycle 2 Total Health Score,Rating Cycle 3 Standard Health Survey Date,Rating Cycle 3 Total Number of Health Deficiencies,Rating Cycle 3 Number of Standard Health Deficiencies,Rating Cycle 3 Number of Complaint Health Deficiencies,Rating Cycle 3 Health Deficiency Score,Rating Cycle 3 Number of Health Revisits,Rating Cycle 3 Health Revisit Score,Rating Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Latitude,Longitude,Geocoding Footnote,Processing Date
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,57,50.0,NaN,Medicare and Medicaid,N,"BURNS NURSING HOME, INC.",1969-09-01,NaN,NaN,N,NaN,N,N,N,Resident,Yes,2.0,NaN,2.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,2.61906,0.42971,1.32259,1.75231

,ZIP Code,Telephone Number,Provider SSA County Code,Number of Certified Beds,Average Number of Residents per Day,Average Number of Residents per Day Footnote,Affiliated Entity ID,Overall Rating,Overall Rating Footnote,Health Inspection Rating,Health Inspection Rating Footnote,QM Rating,QM Rating Footnote,Long-Stay QM Rating,Long-Stay QM Rating Footnote,Short-Stay QM Rating,Short-Stay QM Rating Footnote,Staffing Rating,Staffing Rating Footnote,Reported Staffing Footnote,Physical Therapist Staffing Footnote,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Total number of nurse staff hours per resident per day on the weekend,Registered Nurse hours per resident per day on the weekend,Reported Physical Therapist Staffing Hours per Resident Per Day,Total nursing staff turnover,Total nursing staff turnover footnote,Registered Nurse turnover,Registered Nurse turnover footnote,Number of administrators who have left the nursing home,Administrator turnover footnote,Nursing Case-Mix Index,Nursing Case-Mix Index Ratio,Case-Mix Nurse Aide Staffing Hours per Resident per Day,Case-Mix LPN Staffing Hours per Resident per Day,Case-Mix RN Staffing Hours per Resident per Day,Case-Mix Total Nurse Staffing Hours per Resident per Day,Case-Mix Weekend Total Nurse Staffing Hours per Resident per Day,Adjusted Nurse Aide Staffing Hours per Resident per Day,Adjusted LPN Staffing Hours per Resident per Day,Adjusted RN Staffing Hours per Resident per Day,Adjusted Total Nurse Staffing Hours per Resident per Day,Adjusted Weekend Total Nurse Staffing Hours per Resident per Day,Rating Cycle 1 Total Number of Health Deficiencies,Rating Cycle 1 Number of Standard Health Deficiencies,Rating Cycle 1 Number of Complaint Health Deficiencies,Rating Cycle 1 Health Deficiency Score,Rating Cycle 1 Number of Health Revisits,Rating Cycle 1 Health Revisit Score,Rating Cycle 1 Total Health Score,Rating Cycle 2 Total Number of Health Deficiencies,Rating Cycle 2 Number of Standard Health Deficiencies,Rating Cycle 2 Number of Complaint Health Deficiencies,Rating Cycle 2 Health Deficiency Score,Rating Cycle 2 Number of Health Revisits,Rating Cycle 2 Health Revisit Score,Rating Cycle 2 Total Health Score,Rating Cycle 3 Total Number of Health Deficiencies,Rating Cycle 3 Number of Standard Health Deficiencies,Rating Cycle 3 Number of Complaint Health Deficiencies,Rating Cycle 3 Health Deficiency Score,Rating Cycle 3 Number of Health Revisits,Rating Cycle 3 Health Revisit Score,Rating Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Latitude,Longitude,Geocoding Footnote
count,14817.000000,1.481700e+04,14817.000000,14817.000000,14767.000000,50.0,9885.000000,14673.000000,144.000000,14673.000000,144.000000,14563.000000,350.000000,14230.000000,683.000000,12146.000000,2671.000000,14613.000000,993.000000,325.0,325.0,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,13283.000000,1534.0,12231.000000,2586.0,12251.000000,2566.0,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14492.000000,14466.000000,14466.000000,14466.000000,14466.000000,14466.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14760.000000,14672.000000,14672.000000,14672.000000,14672.000000,14672.000000,14672.000000,14672.000000,14760.000000,14817.000000,14817.000000,7830.000000,14817.000000,1.481700e+04,14817.000000,14817.000000,14817.000000,14817.000000,797.0
mean,5034

## Server

### Data

In [14]:
class Data:
    def __init__(self, *name, data):
        self.name = name
        self.data = data

In [ ]:
# Emulates the API of what the server will be
# Server has API to take in data, process it, and return the geometry to display

server = auto.types.SimpleNamespace()
server.buildings = BuildingsManager(path=config.buildings.gendir/'buildings.csv')
server.roads = Roads('zip:///mnt/seenas2/data/byod-roads-shapefile.zip')
# building_manager = BuildingsManager(path=config.buildings.gendir/'buildings.csv')
# roads = Roads('zip:///mnt/seenas2/data/byod-roads-shapefile.zip')


# server.map_data = map_data

Opening data/gen/buildings.csv...Done.


In [ ]:
server.data = {}
server.colormaps = {
    'viridis': Colormap('viridis'),
    'oranges': Colormap('Oranges'),
    'cool': Colormap('cool'),
}


In [ ]:

# Client: Displays the returned data from the server
class ComvisClient:
    def __init__(self):
        self.lmap = None

    def visualize(self, *, loc, radius: float):
        lat, lon = MapUtil.addr_to_coords(loc)
        self.lmap = Map(center=(lat, lon), zoom=MapUtil.zoom_from_radius(radius_miles=radius))
        self.lmap.use_mapbox('satellite', config.map.token)
        self.lmap.display()


In [ ]:
def scope():
    /display providers_data
    cv = ComvisClient()

/scope

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Telephone Number,Provider SSA County Code,County/Parish,Ownership Type,...,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Latitude,Longitude,Geocoding Footnote,Processing Date
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,...,NaN,1,23989.00,0,1,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",34.5149,-87.736,NaN,2024-09-01
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,For profit - Corporation,...,0.0,0,0.00,0,0,"260 WEST WALNUT STREET,SYLACAUGA,AL,35150",33.1637,-86.254,NaN,2024-09-01
2,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,Government - County,...,NaN,0,0.00,0,0,"380 WOODS COVE ROAD,SCOTTSBORO,AL,35768",34.6611,-86.047,NaN,2024-09-01
3,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,For profit - Individual,...,0.0,0,0.00,0,0,"7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206",33.5595,-86.722,NaN,2024-09-01
4,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,360,Jefferson,For profit - Individual,...,NaN,0,0.00,0,0,"6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111",33.3221,-87.034,NaN,2024-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14812,745022,WARE MEMORIAL CARE CENTER,1510 S. VAN BUREN ST.,AMARILLO,TX,79101,8063730471,860,Potter,For profit - Corporation,...,NaN,0,0.00,0,0,"1510 S. VAN BUREN ST.,AMARILLO,TX,79101",35.1987,-101.840,NaN,2024-09-01
14813,745038,TIERRA ESTE NURSING AND REHABILITATION CENTER,14300 PEBBLE HILLS BLVD,EL PASO,TX,79938,9159559998,480,El Paso,For profit - Corporation,...,NaN,7,31752.27,0,7,"14300 PEBBLE HILLS BLVD,EL PASO,TX,79938",31.7822,-106.230,NaN,2024-09-01
14814,745039,MIDTOWNE MEADOWS HEALTH AND REHAB,110 DYLAN WAY,MIDLOTHIAN,TX,76065,1111,470,Ellis,For profit - Limited Liability company,...,NaN,5,20131.64,0,5,"110 DYLAN WAY,MIDLOTHIAN,TX,76065",32.4783,-96.982,22.0,2024-09-01
14815,745040,THE SARAH ROBERTS FRENCH HOME,1315 TEXAS AVE,SAN ANTONIO,TX,78201,2107364238,130,Bexar,Non profit - Corporation,...,NaN,7,35733.13,0,7,"1315 TEXAS AVE,SAN ANTONIO,TX,78201",29.4494,-98.534,NaN,2024-09-01


NameError: name 'ComvisClient' is not defined